#  Linear advection equation with source term

This file simulates the problem shown in Figure 17.3.

$$
q_t + a q_x = -\beta(x) q, \qquad q(x,0) = q_0(x), \qquad \beta(x) = \begin{cases}
1 & x < 0 \\
1 - x & 0 \le x \le 1 \\
0 & x > 1 \end{cases}
$$

The exact solution is

$$
q(x,t) = \exp\left(-\frac{1}{a} \int_{x-at}^x \beta(s) ds \right) q_0(x-at)
$$

We use $a=1$ in the computations.

In [ ]:
%config InlineBackend.figure_format = 'svg'
from clawpack import pyclaw
from clawpack import riemann
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def beta(x):
    return ((x < 0.0)*(1.0)
            + (0.0 <= x)*(x <= 1.0)*(1.0 - x)
            + (x > 1.0)*(0.0))

# Exact solution, also used for initial condition
def uexact(x,t):
    alpha, x0 = 200.0, 0.25
    xx = x - t
    b1 = (0.0 - xx) + (x - 0.5 * x**2)
    b2 = x - xx - 0.5 * (x**2 - xx**2)
    bint = (xx <= 0.0) * b1 + (xx > 0.0) * b2
    return np.exp(-bint) * np.exp(-alpha * (xx-x0)**2)

This function calls clawpack to solve the problem.

In [ ]:
# Exact solution of source ODE
def step_source(solver,state,dt):
    x = state.grid.x.centers
    state.q[0,:] *= np.exp(-beta(x) * dt)

def solve(ncell=50, order=1, limiter=0, source_split=1):
    claw = pyclaw.Controller()
    claw.tfinal = 0.5
    claw.keep_copy = True       # Keep solution data in memory for plotting
    claw.output_format = None   # Don't write solution data to file
    claw.num_output_times = 10  # Write 50 output frames
    claw.verbosity = 0

    riemann_solver = riemann.advection_1D
    claw.solver = pyclaw.ClawSolver1D(riemann_solver)
    claw.solver.all_bcs = pyclaw.BC.extrap
    claw.solver.dt_initial = 1.e99
    claw.solver.order = order
    claw.solver.limiters = limiter
    claw.solver.source_split = source_split
    claw.solver.step_source = step_source

    xmin, xmax = 0.0, 1.0
    domain = pyclaw.Domain( (xmin,), (xmax,), (ncell,))

    claw.solution = pyclaw.Solution(claw.solver.num_eqn, domain)

    x = domain.grid.x.centers
    claw.solution.q[0, :] = uexact(x, 0.0)

    claw.solution.state.problem_data['u'] = 1.0

    status = claw.run()

    return claw


Plot initial solution and at a later time.

In [ ]:
def make_plots(claw):
    x = claw.grid.x.centers
    q0 = claw.frames[0].q[0, :]
    q = claw.frames[-1].q[0, :]
    plt.plot(x, q0, '-s', label='Initial')
    plt.plot(x, q, '-o', fillstyle='none', label='Final')
    plt.xlabel('x'); plt.ylabel('q')
    plt.title('Time = ' + str(claw.frames[-1].t))
    plt.legend()

## First order (upwind) with Strang splitting

In [ ]:
claw1 = solve(order=1, limiter=0, source_split=2)
make_plots(claw1)

## Lax-Wendroff with Godunov splitting

In [ ]:
claw2 = solve(order=2, limiter=0, source_split=1)
make_plots(claw2)

## Lax-Wendroff with Strang splitting

In [ ]:
claw3 = solve(order=2, limiter=0, source_split=2)
make_plots(claw3)

## Compare

In [ ]:
x = claw1.grid.x.centers
t = claw1.frames[-1].t

xe = np.linspace(x[0],x[-1],200)
qe = uexact(xe, t)

q1 = claw1.frames[-1].q[0, :]
q2 = claw2.frames[-1].q[0, :]
q3 = claw3.frames[-1].q[0, :]
plt.plot(xe, qe, '-', fillstyle='none', label='Exact')
plt.plot(x, q1, 'o', fillstyle='none', label='Upwind+Strang')
plt.plot(x, q2, 's', fillstyle='none', label='LW+Godunov')
plt.plot(x, q3, '*', fillstyle='none', label='LW+Strang')
plt.xlim(0.5, 1.0)
plt.xlabel('x'); plt.ylabel('q')
plt.title('Time = ' + str(t))
plt.legend();